In [1]:
import torch
from train import train_model
from dataset.utils import *
from utils import *
import numpy as np
import pickle
import time

In [2]:
then=time.time()

In [3]:
batch_size = 32
def redefine_train_parameters():
    global device, epochs, learning_rate, loss_lr, num_classes, embedding_size
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    epochs = 20
    learning_rate = 0.001
    loss_lr = 0.01
    try:
        num_classes = get_num_classes(dataset)
    except:
        num_classes = get_num_classes(train_dataset)
    embedding_size = 512

In [4]:
def train(train_dataset, test_dataset):
    train_loader = load_dataset(train_dataset, batch_size, shuffle=True)
    test_loader = load_dataset(test_dataset, batch_size, shuffle=False)
    model = train_model(model_name='resnet50',
                    loss_func_name='proxy_anchor',
                    num_classes=num_classes,
                    dataloader=train_loader,
                    epochs=epochs,
                    learning_rate=learning_rate,
                    verbose=0)
    return model, train_loader, test_loader

In [5]:
def cross_validate(dataset, k=10):
    np.random.shuffle(np.array(dataset))
    k=k
    split_len = len(dataset)//k
    max_index = split_len*k
    r1,r2,r4,r8,r16,r32= [],[],[],[],[],[]
    for fold in range(k):
        print(f'''----------
fold {fold+1}:''')
        start = split_len * fold
        train_dataset = dataset[:start] + dataset[start + split_len:max_index]
        test_dataset = dataset[start:start+split_len]
        train_loader = load_dataset(train_dataset, batch_size, shuffle=True)
        test_loader = load_dataset(test_dataset, batch_size, shuffle=False)
        model = train_model(model_name='resnet50',
                        loss_func_name='proxy_anchor',
                        num_classes=num_classes,
                        dataloader=train_loader,
                        epochs=epochs,
                        learning_rate=learning_rate,
                        verbose=0)
        recall = evaluate_cos(model, test_loader)
        r1.append(recall[0])
        r2.append(recall[1])
        r4.append(recall[2])
        r8.append(recall[3])
        r16.append(recall[4])
        r32.append(recall[5])
    return r1,r2,r4,r8,r16,r32

In [6]:
def avg_recall(name):
    print(f'''Average Recall {name}:
R@1 : {np.mean(r1)}
R@2 : {np.mean(r2)}
R@4 : {np.mean(r4)}
R@8 : {np.mean(r8)}
R@16 : {np.mean(r16)}
R@32 : {np.mean(r32)}''')
    
    print(f'''Average Recall {name}:
R@1 : {np.mean(r1)*100:.4f}
R@2 : {np.mean(r2)*100:.4f}
R@4 : {np.mean(r4)*100:.4f}
R@8 : {np.mean(r8)*100:.4f}
R@16 : {np.mean(r16)*100:.4f}
R@32 : {np.mean(r32)*100:.4f}''')

In [7]:
def save_output(output, name):
    try:
        data = load_output()
    except:
        data = dict()
    data[name] = output
    with open('recall50_20.pkl', 'wb') as file:
        pickle.dump(data, file)
        
def load_output():
    with open('recall50_20.pkl', 'rb') as file:
        return pickle.load(file)

In [9]:
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, face_detection=False)
redefine_train_parameters()
name=f'PA'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_745932/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_745932/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:


/workspace/Experiments/Clean/dataset/utils.py:119: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  except: num_classes = len(Counter(np.array(dataset, dtype=object)[:,1]))


Finished Training, Time: 40.2199 Best loss: 2.5464


100%|██████████| 8/8 [00:00<00:00, 121.44it/s]


...

R@1 : 85.317
R@2 : 90.476
R@4 : 93.651
R@8 : 97.222
R@16 : 98.810
R@32 : 99.603
----------
fold 9:
Finished Training, Time: 40.3167 Best loss: 2.2924


100%|██████████| 8/8 [00:00<00:00, 114.02it/s]


R@1 : 88.095
R@2 : 92.063
R@4 : 96.032
R@8 : 96.429
R@16 : 98.810
R@32 : 99.603
----------
fold 10:
Finished Training, Time: 39.9203 Best loss: 2.3906


100%|██████████| 8/8 [00:00<00:00, 122.65it/s]

R@1 : 80.952
R@2 : 87.698
R@4 : 93.254
R@8 : 96.825
R@16 : 96.825
R@32 : 98.810


In [12]:
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, face_detection=False, autocontrast=True)
redefine_train_parameters()
name=f'PA+AutoContrast'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_745932/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_745932/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 39.9901 Best loss: 2.3822


100%|██████████| 8/8 [00:00<00:00, 121.80it/s]


R@1 : 83.333
R@2 : 90.079
R@4 : 94.841
R@8 : 97.619
R@16 : 99.206
R@32 : 100.000
----------
fold 2:
Finished Training, Time: 40.1387 Best loss: 2.5073


100%|██████████| 8/8 [00:00<00:00, 122.13it/s]


...

R@1 : 86.508
R@2 : 90.476
R@4 : 94.841
R@8 : 97.222
R@16 : 98.810
R@32 : 99.206
----------
fold 9:
Finished Training, Time: 40.3935 Best loss: 2.5995


100%|██████████| 8/8 [00:00<00:00, 121.32it/s]


R@1 : 86.905
R@2 : 91.270
R@4 : 96.032
R@8 : 97.619
R@16 : 98.810
R@32 : 100.000
----------
fold 10:
Finished Training, Time: 40.6756 Best loss: 2.3341


100%|██████████| 8/8 [00:00<00:00, 120.21it/s]


R@1 : 84.127
R@2 : 88.492
R@4 : 92.063
R@8 : 97.222
R@16 : 98.413
R@32 : 99.206


In [10]:
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100)
redefine_train_parameters()
name=f'PA+FaceDetection'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_745932/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_745932/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 40.2109 Best loss: 2.5159


100%|██████████| 8/8 [00:00<00:00, 121.34it/s]


R@1 : 82.143
R@2 : 86.905
R@4 : 92.063
R@8 : 94.841
R@16 : 98.016
R@32 : 99.206
----------
fold 2:
Finished Training, Time: 40.4343 Best loss: 2.1858


100%|██████████| 8/8 [00:00<00:00, 111.67it/s]


...

R@1 : 82.143
R@2 : 90.079
R@4 : 95.635
R@8 : 97.619
R@16 : 99.206
R@32 : 99.603
----------
fold 9:
Finished Training, Time: 40.8435 Best loss: 2.6184


100%|██████████| 8/8 [00:00<00:00, 119.90it/s]


R@1 : 83.333
R@2 : 89.683
R@4 : 94.841
R@8 : 97.619
R@16 : 99.603
R@32 : 100.000
----------
fold 10:
Finished Training, Time: 40.3612 Best loss: 2.6459


100%|██████████| 8/8 [00:00<00:00, 120.67it/s]


R@1 : 78.968
R@2 : 84.921
R@4 : 88.889
R@8 : 94.841
R@16 : 98.016
R@32 : 99.206


In [11]:
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, face_detection=False, crop=125)
redefine_train_parameters()
name=f'PA+Crop(125)'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_745932/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_745932/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 40.1747 Best loss: 2.5179


100%|██████████| 8/8 [00:00<00:00, 122.57it/s]


R@1 : 90.079
R@2 : 94.841
R@4 : 95.238
R@8 : 96.429
R@16 : 98.016
R@32 : 99.603
----------
fold 2:
Finished Training, Time: 39.9077 Best loss: 2.3856


100%|██████████| 8/8 [00:00<00:00, 121.61it/s]


...

R@1 : 82.143
R@2 : 86.905
R@4 : 90.476
R@8 : 95.635
R@16 : 98.016
R@32 : 99.206
----------
fold 9:
Finished Training, Time: 40.2698 Best loss: 2.5600


100%|██████████| 8/8 [00:00<00:00, 122.07it/s]


R@1 : 87.698
R@2 : 92.063
R@4 : 94.841
R@8 : 97.222
R@16 : 98.810
R@32 : 99.603
----------
fold 10:
Finished Training, Time: 40.1167 Best loss: 2.5392


100%|██████████| 8/8 [00:00<00:00, 122.68it/s]


R@1 : 86.905
R@2 : 90.476
R@4 : 93.254
R@8 : 96.429
R@16 : 98.016
R@32 : 99.603


In [19]:
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, face_detection=False, crop=125, autocontrast=True)
redefine_train_parameters()
name=f'PA+AutoContrast+Crop(125)'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_745932/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_745932/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 40.0707 Best loss: 2.5380


100%|██████████| 8/8 [00:00<00:00, 122.15it/s]


R@1 : 87.302
R@2 : 90.873
R@4 : 94.841
R@8 : 97.619
R@16 : 98.810
R@32 : 99.603
----------
fold 2:
Finished Training, Time: 39.9147 Best loss: 2.3488


100%|██████████| 8/8 [00:00<00:00, 121.78it/s]


...

R@1 : 85.317
R@2 : 91.667
R@4 : 94.048
R@8 : 96.825
R@16 : 98.016
R@32 : 98.413
----------
fold 9:
Finished Training, Time: 41.2006 Best loss: 2.3931


100%|██████████| 8/8 [00:00<00:00, 120.73it/s]


R@1 : 84.921
R@2 : 90.873
R@4 : 95.238
R@8 : 96.825
R@16 : 98.810
R@32 : 99.603
----------
fold 10:
Finished Training, Time: 40.9426 Best loss: 2.4150


100%|██████████| 8/8 [00:00<00:00, 120.73it/s]


R@1 : 80.159
R@2 : 88.095
R@4 : 94.444
R@8 : 96.429
R@16 : 98.413
R@32 : 99.603


In [13]:
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, face_detection=False, crop=125, median_unsharp=True, median_unsharp_size=3, median_unsharp_amount=2, median=True, median_size=3)
redefine_train_parameters()
name=f'PA+Crop(125)+MedianUnsharp(size=3,amount=2)+Median(size=3)'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_745932/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_745932/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 41.8469 Best loss: 2.3539


100%|██████████| 8/8 [00:00<00:00, 107.36it/s]


R@1 : 86.508
R@2 : 91.270
R@4 : 95.238
R@8 : 97.619
R@16 : 98.413
R@32 : 99.603
----------
fold 2:
Finished Training, Time: 41.9167 Best loss: 2.6002


100%|██████████| 8/8 [00:00<00:00, 104.42it/s]


...

R@1 : 83.730
R@2 : 88.889
R@4 : 92.460
R@8 : 95.635
R@16 : 97.222
R@32 : 98.413
----------
fold 9:
Finished Training, Time: 42.0813 Best loss: 2.4576


100%|██████████| 8/8 [00:00<00:00, 104.99it/s]


R@1 : 89.286
R@2 : 92.857
R@4 : 97.222
R@8 : 98.413
R@16 : 99.206
R@32 : 99.603
----------
fold 10:
Finished Training, Time: 42.5043 Best loss: 2.5642


100%|██████████| 8/8 [00:00<00:00, 105.16it/s]


R@1 : 86.508
R@2 : 91.270
R@4 : 96.032
R@8 : 98.016
R@16 : 99.206
R@32 : 99.206


In [20]:
# [(1,i) for i in range(1,10,2)]+
mu_amount_size = [(2,i) for i in range(1,6,2)]+[(3,i) for i in range(1,4,2)]
msizes=[1,3,5]
for muamount, musize in mu_amount_size:
    for msize in msizes:
        dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, face_detection=False, crop=125, median_unsharp=True, median_unsharp_size=musize, median_unsharp_amount=muamount, median_hsv=True, median_hsv_method='sv', median_hsv_size=msize)
        redefine_train_parameters()
        name=f'PA+Crop(125)+MedianUnsharp(size={musize},amount={muamount})+MedianHSV(method=sv,size={msize})'
        recalls=cross_validate(dataset)
        save_output(recalls,f'{name}')
        save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_745932/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_745932/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 42.4475 Best loss: 2.5813


100%|██████████| 8/8 [00:00<00:00, 106.42it/s]


R@1 : 89.286
R@2 : 94.444
R@4 : 97.222
R@8 : 98.413
R@16 : 99.603
R@32 : 100.000
----------
fold 2:
Finished Training, Time: 42.9009 Best loss: 2.2317


100%|██████████| 8/8 [00:00<00:00, 77.25it/s]


...

100%|██████████| 8/8 [00:00<00:00, 107.05it/s]


R@1 : 90.873
R@2 : 93.651
R@4 : 95.238
R@8 : 97.222
R@16 : 98.016
R@32 : 99.603
----------
fold 7:
Finished Training, Time: 42.1943 Best loss: 2.2924


100%|██████████| 8/8 [00:00<00:00, 105.35it/s]


R@1 : 84.921
R@2 : 88.492
R@4 : 93.254
R@8 : 96.825
R@16 : 99.206
R@32 : 100.000
----------
fold 8:


KeyboardInterrupt: 

In [46]:
musize,muamount,msize=7,1,5
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, autocontrast=True, face_detection=False, crop=125, median_unsharp=True, median_unsharp_size=musize, median_unsharp_amount=muamount, median_hsv=True, median_hsv_method='sv', median_hsv_size=msize)
redefine_train_parameters()
name=f'PA+Crop(125)+AutoContrast+MedianUnsharp(size={musize},amount={muamount})+MedianHSV(method=sv,size={msize})'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_748950/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_748950/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 42.4190 Best loss: 2.4319


100%|██████████| 8/8 [00:00<00:00, 104.70it/s]


R@1 : 88.492
R@2 : 94.444
R@4 : 96.032
R@8 : 97.222
R@16 : 99.603
R@32 : 99.603
----------
fold 2:
Finished Training, Time: 41.9608 Best loss: 2.2480


100%|██████████| 8/8 [00:00<00:00, 71.22it/s]


...

R@1 : 80.952
R@2 : 86.111
R@4 : 92.063
R@8 : 95.238
R@16 : 98.016
R@32 : 99.206
----------
fold 9:
Finished Training, Time: 42.0649 Best loss: 2.4688


100%|██████████| 8/8 [00:00<00:00, 105.15it/s]


R@1 : 88.095
R@2 : 90.476
R@4 : 92.857
R@8 : 96.429
R@16 : 99.206
R@32 : 100.000
----------
fold 10:
Finished Training, Time: 43.4615 Best loss: 2.5631


100%|██████████| 8/8 [00:00<00:00, 102.67it/s]


R@1 : 84.127
R@2 : 90.079
R@4 : 95.238
R@8 : 97.222
R@16 : 99.206
R@32 : 100.000


In [47]:
musize,muamount,msize=3,2,1
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, autocontrast=True, face_detection=False, crop=125, median_unsharp=True, median_unsharp_size=musize, median_unsharp_amount=muamount, median_hsv=True, median_hsv_method='sv', median_hsv_size=msize)
redefine_train_parameters()
name=f'PA+Crop(125)+AutoContrast+MedianUnsharp(size={musize},amount={muamount})+MedianHSV(method=sv,size={msize})'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_748950/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_748950/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 41.8056 Best loss: 2.3795


100%|██████████| 8/8 [00:00<00:00, 106.26it/s]


R@1 : 90.873
R@2 : 93.651
R@4 : 94.841
R@8 : 97.619
R@16 : 98.413
R@32 : 99.603
----------
fold 2:
Finished Training, Time: 42.3046 Best loss: 2.4753


100%|██████████| 8/8 [00:00<00:00, 104.09it/s]


...

R@1 : 81.349
R@2 : 89.286
R@4 : 94.444
R@8 : 96.825
R@16 : 98.016
R@32 : 98.810
----------
fold 9:
Finished Training, Time: 42.2925 Best loss: 2.3528


100%|██████████| 8/8 [00:00<00:00, 99.26it/s]


R@1 : 86.508
R@2 : 93.254
R@4 : 94.841
R@8 : 97.222
R@16 : 98.413
R@32 : 100.000
----------
fold 10:
Finished Training, Time: 41.9700 Best loss: 2.4200


100%|██████████| 8/8 [00:00<00:00, 105.97it/s]


R@1 : 84.524
R@2 : 90.079
R@4 : 92.460
R@8 : 95.238
R@16 : 97.619
R@32 : 98.810


In [48]:
musize,muamount,msize=3,2,3
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, autocontrast=True, face_detection=False, crop=125, median_unsharp=False, median_unsharp_size=musize, median_unsharp_amount=muamount, median_hsv=True, median_hsv_method='sv', median_hsv_size=msize)
redefine_train_parameters()
name=f'PA+Crop(125)+AutoContrast+MedianHSV(method=sv,size={msize})'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')

/tmp/ipykernel_748950/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_748950/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 42.2208 Best loss: 2.5069


100%|██████████| 8/8 [00:00<00:00, 103.57it/s]


R@1 : 84.921
R@2 : 91.270
R@4 : 95.635
R@8 : 98.413
R@16 : 98.810
R@32 : 99.603
----------
fold 2:
Finished Training, Time: 42.0221 Best loss: 2.4669


100%|██████████| 8/8 [00:00<00:00, 106.26it/s]


...

R@1 : 79.762
R@2 : 86.905
R@4 : 93.651
R@8 : 96.032
R@16 : 99.206
R@32 : 99.206
----------
fold 9:
Finished Training, Time: 42.3339 Best loss: 2.5584


100%|██████████| 8/8 [00:00<00:00, 104.93it/s]


R@1 : 86.905
R@2 : 91.667
R@4 : 94.444
R@8 : 98.413
R@16 : 99.206
R@32 : 100.000
----------
fold 10:
Finished Training, Time: 42.1196 Best loss: 2.5360


100%|██████████| 8/8 [00:00<00:00, 103.89it/s]


R@1 : 87.302
R@2 : 91.270
R@4 : 95.238
R@8 : 96.825
R@16 : 98.413
R@32 : 99.206


In [44]:
dataset = make_dataset('LFWCustom', split='10fold', max_classes=50, min_samples=20, image_size=100, pad_ratio=1.3, face_detection_crop=140)
redefine_train_parameters()
name=f'PA+FaceDetection(pad_ratio=1.3,crop=140))'
recalls=cross_validate(dataset)
save_output(recalls,f'{name}')
save_output([np.mean(arr) for arr in recalls],f'{name}_mean')


/tmp/ipykernel_748950/3704652543.py:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np.random.shuffle(np.array(dataset))
/tmp/ipykernel_748950/3704652543.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.shuffle(np.array(dataset))


----------
fold 1:
Finished Training, Time: 40.1760 Best loss: 2.3954


100%|██████████| 8/8 [00:00<00:00, 121.56it/s]


R@1 : 82.540
R@2 : 88.095
R@4 : 93.254
R@8 : 96.429
R@16 : 98.016
R@32 : 99.206
----------
fold 2:
Finished Training, Time: 40.2359 Best loss: 2.4664


100%|██████████| 8/8 [00:00<00:00, 123.06it/s]


...

R@1 : 81.349
R@2 : 88.889
R@4 : 91.270
R@8 : 92.460
R@16 : 96.429
R@32 : 98.413
----------
fold 9:
Finished Training, Time: 40.0158 Best loss: 2.4658


100%|██████████| 8/8 [00:00<00:00, 121.98it/s]


R@1 : 84.524
R@2 : 89.286
R@4 : 92.857
R@8 : 95.635
R@16 : 98.016
R@32 : 99.206
----------
fold 10:
Finished Training, Time: 40.8661 Best loss: 2.5267


100%|██████████| 8/8 [00:00<00:00, 121.44it/s]

R@1 : 78.175
R@2 : 88.889
R@4 : 91.667
R@8 : 95.635
R@16 : 96.429
R@32 : 100.000


In [9]:
total_time = time.time()-then
save_output(total_time, 'total_time')

In [10]:
load_output()['total_time']

1840.0826926231384

In [45]:
{key:val for key,val in load_output().items() if name in key}

{'PA+FaceDetection(pad_ratio=1.3,crop=140))': ([0.8253968253968254,
   0.9007936507936508,
   0.9325396825396826,
   1.0,
   1.0,
   0.9484126984126984,
   0.7936507936507936,
   0.8134920634920635,
   0.8452380952380952,
   0.7817460317460317],
  [0.8809523809523809,
   0.9285714285714286,
   0.9682539682539683,
   1.0,
   1.0,
   0.9603174603174603,
   0.8968253968253969,
   0.8888888888888888,
   0.8928571428571429,
   0.8888888888888888],
  [0.9325396825396826,
   0.9603174603174603,
   0.9841269841269841,
   1.0,
   1.0,
   0.9801587301587301,
   0.9404761904761905,
   0.9126984126984127,
   0.9285714285714286,
   0.9166666666666666],
  [0.9642857142857143,
   0.9761904761904762,
   0.9920634920634921,
   1.0,
   1.0,
   0.9880952380952381,
   0.9761904761904762,
   0.9246031746031746,
   0.9563492063492064,
   0.9563492063492064],
  [0.9801587301587301,
   0.9880952380952381,
   0.9920634920634921,
   1.0,
   1.0,
   0.9920634920634921,
   0.9880952380952381,
   0.964285714285714

In [ ]:
{key:[np.mean(x) for x in val] for key,val in load_output().items() if 'HSV(method=sv' in key and 'mean' in key}